In [5]:
import sys
sys.executable

'/usr/workspace/wsa/jones289/miniconda3/envs/atomsci/bin/python'

In [6]:
import pandas as pd
import deepchem as dc
from atomsci.ddm.pipeline.splitting import TrainValidTestSplitting
import pickle
from argparse import Namespace
print(dc.__version__)

2.5.0


In [7]:
def load_dataset():
    
    df = pd.read_csv("/g/g13/jones289/workspace/hd-cuda-master/datasets/pdbbind_fingerprints/pdbbind_2019_fps_new/v_2019_full_pdbid_list.csv", index_col=0)
#     df = pd.read_csv('/usr/WS1/jones289/hd-cuda-master/datasets/pdbbind_fingerprints/pdbbind_2016_fps_new/v_2016_refined_pdbid_list.csv', index_col=0)
    df = df[df['label'].apply(lambda x: x != 2)]
#     print(df.head())
    dc_dataset = dc.data.NumpyDataset(X=df[[x for x in df.columns if x not in ['label', 'pdbid', 'smiles']]], y=df['label'], ids=df['pdbid'])
    df['compound_id'] = df['pdbid']
    return dc_dataset, df[['label', 'compound_id', 'smiles']]


def main():
    pdbbind_dataset, attr_df = load_dataset()
    print(pdbbind_dataset)
    print(pdbbind_dataset.ids)
    print(pdbbind_dataset.y)

    params_dict = {'split': 'ave_min',
               'splitter': 'ave_min',
                 'featurizer': 'ecfp',
                 'id_col': 'compound_id',
              'split_test_frac': 0.2,
              'split_valid_frac': 0.1}
    params = Namespace(**params_dict)

    ave_splitter = TrainValidTestSplitting(params=params)
    result = ave_splitter.split_dataset(dataset=pdbbind_dataset, attr_df=attr_df, smiles_col='smiles')
    with open('ave_split_result.pkl', 'wb') as handle:
        pickle.dump(result, handle)

main()

FileNotFoundError: [Errno 2] No such file or directory: '/g/g13/jones289/workspace/hd-cuda-master/datasets/pdbbind_fingerprints/pdbbind_2019_fps_new/v_2019_full_pdbid_list.csv'

In [ ]:
import 

In [ ]:
with open('ave_split_result.pkl', 'rb') as handle:
    data = pickle.load(handle)
    data

In [ ]:
data

In [ ]:
train_d = data[0][0][0]
val_d = data[0][0][1]
test_d = data[1]

In [ ]:
def dump_dataset_to_disk_as_pickle(dataset, file_prefix):
    print(dataset.X.shape, dataset.y.shape, dataset.ids.shape)
    pkl_path = file_prefix + '.pkl'
    csv_path = file_prefix + '.csv'
    
    id_df = pd.DataFrame({'pdbid': dataset.ids})
    id_df.to_csv(csv_path, index=False)
    
    pkl_data = (dataset.X, dataset.y)
    with open(pkl_path, 'wb') as handle:
        pickle.dump(pkl_data, handle)
    
    

In [ ]:
dump_dataset_to_disk_as_pickle(train_d, "ave_min_split_train")
dump_dataset_to_disk_as_pickle(val_d, "ave_min_split_val")
dump_dataset_to_disk_as_pickle(test_d, "ave_min_split_test")
